In [3]:
import os
from google.colab import drive
drive.mount('/content/drive')

path = "/content/drive/My Drive"

os.chdir(path)
os.listdir(path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


['至大三上成绩单.pdf',
 'BB9E1FA0@D3815A65.36CC2E59',
 '体检信息登记表.xlsx',
 'resume.pdf',
 '申请信息表--推荐信表格.gdoc',
 '脚手架.rar',
 'Colab Notebooks',
 'Resume_xsy_2020_for job.pdf',
 'Untitled0.ipynb',
 'extracted_dialogue.txt']

In [5]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
SOS_token = 0
EOS_token = 1
learning_rate = 0.1
hidden_size = 256
teacher_forcing_ratio = 0.5



class Lang:
    def __init__(self, name="dict"):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {EOS_token:"EOS",SOS_token:"SOS"}
        self.n_words = 2  

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)  
        self.gru = nn.GRU(hidden_size, hidden_size) 

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1) 
        output, hidden = self.gru(embedded, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

def prepareData():
    lang_dict = Lang()
    sentences = []
    max_length=0
    with open("./extracted_dialogue.txt","r") as f:
        lines = f.read().strip().split("\n")
        for i in range(0, len(lines), 2):
            tmp = lines[i].strip()
            sentences.append(tmp)
            lang_dict.addSentence(tmp)
            if len(tmp) > max_length:
                max_length = len(tmp)
    return lang_dict, sentences, max_length

def getSentenceTensor(lang, sentence):
    indexes = [lang.word2index[word] for word in sentence.split(' ')]
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)

def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length):
    
    encoder_hidden = encoder.initHidden() 

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  

    else:
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach() 

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length


def trainIters(encoder, decoder, lang_dict, max_length, sentences,  n_iters, print_every=1000, learning_rate=0.01):
    print_loss_total = 0

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_data = [getSentenceTensor(lang_dict, random.choice(sentences))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        iter_data = training_data[iter - 1]
        input_tensor = iter_data
        target_tensor = iter_data

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion, max_length)
        print_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('iter, loss =  (%d %.4f) ' %  (iter,  print_loss_avg))



lang_dict, sentences, max_length = prepareData()
print(max_length)

encoder = EncoderRNN(lang_dict.n_words, hidden_size).to(device)
decoder = DecoderRNN(hidden_size, lang_dict.n_words).to(device)

trainIters(encoder, decoder, lang_dict, max_length, sentences, 10000, print_every=100)



1903
iter, loss =  (100 6.7820) 
iter, loss =  (200 7.1964) 
iter, loss =  (300 5.9220) 
iter, loss =  (400 5.9519) 
iter, loss =  (500 5.7178) 
iter, loss =  (600 6.0532) 
iter, loss =  (700 6.3610) 
iter, loss =  (800 6.2912) 
iter, loss =  (900 6.0389) 
iter, loss =  (1000 6.1918) 
iter, loss =  (1100 5.8251) 
iter, loss =  (1200 5.4222) 
iter, loss =  (1300 6.1846) 
iter, loss =  (1400 5.3709) 
iter, loss =  (1500 5.9080) 
iter, loss =  (1600 5.9293) 
iter, loss =  (1700 5.0843) 
iter, loss =  (1800 6.1497) 
iter, loss =  (1900 5.8890) 
iter, loss =  (2000 5.2503) 
iter, loss =  (2100 5.8960) 
iter, loss =  (2200 5.9615) 
iter, loss =  (2300 5.8921) 
iter, loss =  (2400 5.4792) 
iter, loss =  (2500 5.1385) 
iter, loss =  (2600 5.5372) 
iter, loss =  (2700 5.8608) 
iter, loss =  (2800 5.5288) 
iter, loss =  (2900 5.3126) 
iter, loss =  (3000 5.9882) 
iter, loss =  (3100 5.3571) 
iter, loss =  (3200 5.2260) 
iter, loss =  (3300 5.5278) 
iter, loss =  (3400 5.1773) 
iter, loss =  (350